# Year Over Year Growth Rate

Assume you're given a table containing information about Wayfair user transactions for different products. Write a query to calculate the year-over-year growth rate for the total spent of each product, grouping the results by product ID.

The output should include the year in ascending order, product ID, current year's spend, previous year's spend & year-on-year growth percentage, rounded to 2 decimal places.

# Answer

I'm importing the `user_transactions2` table:

```
CREATE TABLE user_transactions2 (
	transaction_id smallint,
	product_id integer,
	spend numeric(6, 2),
	transaction_date timestamp
);

COPY user_transactions2
FROM '/YourDirectory/user_transactions2.csv'
WITH (FORMAT CSV, HEADER);

SELECT * FROM user_transactions2;
```

<img src = "user_transactions2 Table.png" width = "600" style = "margin:auto"/>

This uses either the `lead()` or `lag()` function -- I'm just not sure which. I'll try out both & show the correct one.

```
WITH transact
AS (
	SELECT *,
		   lag(spend) OVER (PARTITION BY product_id 
		       ORDER BY date_part('year', 
			   transaction_date)) AS prev_spend
	FROM user_transactions2
)
SELECT date_part('year', transaction_date),
	   product_id,
	   spend,
	   prev_spend,
	   round((spend - prev_spend) / prev_spend * 100, 2)
	   	   AS yoy_growth_rate
FROM transact;
```

<img src = "YoY Growth Rate.png" width = "600" style = "margin:auto"/>

Not gonna explain it -- here is the YoY growth rate results.

```
COPY (
	WITH transact
	AS (
		SELECT *,
			   lag(spend) OVER (PARTITION BY product_id 
			       ORDER BY date_part('year', 
				   transaction_date)) AS prev_spend
		FROM user_transactions2
	)
	SELECT date_part('year', transaction_date),
		   product_id,
		   spend,
		   prev_spend,
		   round((spend - prev_spend) / prev_spend * 100, 
		   	   2) AS yoy_growth_rate
	FROM transact
)
TO '/YourDirectory/YoY Growth Rate Result.csv'
WITH (FORMAT CSV, HEADER);
```